## all imports

In [ ]:
import os
import torch
import dataclasses
import math

import typing as tp
import torch.nn as nn

from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data import get_tokenizer
from torch.nn.utils.rnn import pad_sequence

In [ ]:
import wandb
wandb.login()

## config.py

In [ ]:
@dataclasses.dataclass(init=True)
class TranslationConfig:

    data_dir: str = 'data/'

    special_symbols: tp.List[str] = ('<unk>', '<pad>', '<bos>', '<eos>')

    unk_id: int = 0
    pad_id: int = 1
    bos_id: int = 2
    eos_id: int = 3

    batch_size: int = 64

    num_encoder_layers: int = 6
    num_decoder_layers: int = 6
    emb_size: int = 512
    nhead: int = 8
    dim_feedforward: int = 2048
    dropout: float = 0.1

    num_epochs: int = 15
    lr: float = 0.0005

    def __post_init__(self):
        self.src_files: tp.List[str] = [f'{self.data_dir}train.de-en.de', f'{self.data_dir}val.de-en.de']
        self.tgt_files: tp.List[str] = [f'{self.data_dir}train.de-en.en', f'{self.data_dir}val.de-en.en']
        self.test_file: str = f'{self.data_dir}test1.de-en.de'


CONFIG = TranslationConfig()
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [ ]:
DEVICE

## dataset.py

In [ ]:
class MyTokenizer:
    def __init__(
        self, 
        config: TranslationConfig,
        train_data_file: str, 
        val_data_file: str, 
    ):
        """
        Dataset with texts, supporting BPE tokenizer
        :param data_file: txt file containing texts
        :param sp_model_prefix: path prefix to save tokenizer model
        :param vocab_size: sentencepiece tokenizer vocabulary size
        :param normalization_rule_name: sentencepiece tokenizer normalization rule
        :param model_type: sentencepiece class model type
        """
        self.config = config
        self.train_mode = True
        
        train_texts = []
        with open(train_data_file, encoding="utf-8") as file:
            train_texts.extend(file.readlines())
        self.train_texts = train_texts

        val_texts = []
        with open(val_data_file, encoding="utf-8") as file:
            val_texts.extend(file.readlines())
        self.val_texts = train_texts

        self.tokenizer = get_tokenizer(None)

        self.vocab_transform = build_vocab_from_iterator(
            self._yield_tokens(),
            min_freq=1,
            specials=config.special_symbols,
            special_first=True
        )
        self.vocab_transform.set_default_index(self.config.unk_id)

    def _yield_tokens(self) -> tp.List[str]:
        for data_sample in self.train_texts:
            yield self.tokenizer(data_sample)

    def __getitem__(self, item: int) -> tp.Tuple[torch.Tensor, torch.Tensor]:
        if self.train_mode:
            seq = self.train_texts[item]
        else:
            seq = self.val_texts[item]    

        tokens = self.tokenizer(seq)
        token_ids = self.vocab_transform(tokens)

        token_ids = torch.cat((
            torch.tensor([self.config.bos_id], dtype=torch.int64),
            torch.tensor(token_ids, dtype=torch.int64),
            torch.tensor([self.config.eos_id], dtype=torch.int64)
        ))

        return token_ids
    
    def train(self):
        self.train_mode = True
    
    def eval(self):
        self.train_mode = False


class TextDataset(Dataset):
    def __init__(
        self, 
        config: TranslationConfig,
    ):
        self.config = config
        self.src_tokenizer = MyTokenizer(
            config=config,
            train_data_file=config.src_files[0],
            val_data_file=config.src_files[1],
        )
        self.tgt_tokenizer = MyTokenizer(
            config=config,
            train_data_file=config.tgt_files[0], 
            val_data_file=config.tgt_files[1], 
        )

    def __len__(self):
        if self.src_tokenizer.train_mode:
            return len(self.src_tokenizer.train_texts)
        return len(self.src_tokenizer.val_texts)

    def __getitem__(self, item: int) -> tp.Tuple[torch.Tensor, torch.Tensor]:
        return self.src_tokenizer[item], self.tgt_tokenizer[item]

    def text2ids(self, texts: tp.Union[str, tp.List[str]]) -> tp.Union[tp.List[int], tp.List[tp.List[int]]]:
        """
        Encode a text or list of texts as tokenized indices
        :param texts: text or list of texts to tokenize
        :return: encoded indices
        """
        tokens = self.src_tokenizer.tokenizer(texts)
        token_ids = self.src_tokenizer.vocab_transform(tokens)
        token_ids = torch.cat((
            torch.tensor([self.config.bos_id], dtype=torch.int64),
            torch.tensor(token_ids, dtype=torch.int64),
            torch.tensor([self.config.eos_id], dtype=torch.int64)
        ))
        return torch.tensor(token_ids, dtype=torch.int64)

    def ids2text(self, ids: tp.Union[torch.Tensor, tp.List[int], tp.List[tp.List[int]]]) -> tp.Union[str, tp.List[str]]:
        """
        Decode indices as a text or list of tokens
        :param ids: 1D or 2D list (or torch.Tensor) of indices to decode
        :return: decoded texts
        """
        return self.tgt_tokenizer.vocab_transform.lookup_tokens(ids)

    def train(self):
        self.src_tokenizer.train()
        self.tgt_tokenizer.train()
    
    def eval(self):
        self.src_tokenizer.eval()
        self.tgt_tokenizer.eval()

def collate_fn(batch):
    """
    Add padding to batch samples
    """
    src_batch = [sample[0] for sample in batch]
    tgt_batch = [sample[1] for sample in batch]
    src_batch = pad_sequence(src_batch, padding_value=CONFIG.pad_id)
    tgt_batch = pad_sequence(tgt_batch, padding_value=CONFIG.pad_id)
    return src_batch, tgt_batch


## model.py

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(-1. * torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)

        pos_embedding = pos_embedding.unsqueeze(-2)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: torch.Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])


class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: torch.Tensor):
        return self.embedding(tokens.long()) * torch.sqrt(torch.tensor(self.emb_size))


class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int,
                 dropout: float):
        super().__init__()
        self.transformer = nn.Transformer(
            d_model=emb_size,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
        )
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: torch.Tensor,
                trg: torch.Tensor,
                src_mask: torch.Tensor,
                tgt_mask: torch.Tensor,
                src_padding_mask: torch.Tensor,
                tgt_padding_mask: torch.Tensor,
                memory_key_padding_mask: torch.Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: torch.Tensor, src_mask: torch.Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: torch.Tensor, memory: torch.Tensor, tgt_mask: torch.Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)


def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

    src_padding_mask = (src == CONFIG.pad_id).transpose(0, 1)
    tgt_padding_mask = (tgt == CONFIG.pad_id).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask


## main.py

In [ ]:
def train_epoch(model: Seq2SeqTransformer, optimizer, dataset: TextDataset):
    model.train()
    dataset.train()

    losses = 0
    train_dataloader = DataLoader(dataset, batch_size=CONFIG.batch_size, shuffle=True, collate_fn=collate_fn, num_workers=2)

    for src, tgt in tqdm(train_dataloader, desc='training'):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]
        tgt_out = tgt[1:, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        loss = torch.nn.CrossEntropyLoss(ignore_index=CONFIG.pad_id)(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))


def evaluate(model: Seq2SeqTransformer, dataset: TextDataset):
    model.eval()
    dataset.eval()

    losses = 0
    val_dataloader = DataLoader(dataset, batch_size=CONFIG.batch_size, shuffle=False, collate_fn=collate_fn, num_workers=2)

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]
        tgt_out = tgt[1:, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        loss = torch.nn.CrossEntropyLoss(ignore_index=CONFIG.pad_id)(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))


def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == CONFIG.eos_id:
            break
    return ys


def translate(model: torch.nn.Module, src_sentence: str, text2ids, ids2text):
    model.eval()
    src = text2ids(src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=CONFIG.bos_id).flatten()
    return " ".join(ids2text(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")


## train

In [ ]:
dataset = TextDataset(CONFIG)

In [ ]:
transformer = Seq2SeqTransformer(
    num_encoder_layers=CONFIG.num_encoder_layers,
    num_decoder_layers=CONFIG.num_decoder_layers,
    emb_size=CONFIG.emb_size,
    nhead=CONFIG.nhead,
    src_vocab_size=len(dataset.src_tokenizer.vocab_transform),
    tgt_vocab_size=len(dataset.tgt_tokenizer.vocab_transform),
    dim_feedforward=CONFIG.dim_feedforward,
    dropout=CONFIG.dropout
).to(DEVICE)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

optimizer = torch.optim.Adam(transformer.parameters(), lr=CONFIG.lr, betas=(0.9, 0.98), eps=1e-9)
scheduler = None

In [ ]:
wandb.init(
    project="text-translation",
    config=CONFIG.__dict__
)

In [ ]:
for epoch in range(1, CONFIG.num_epochs + 1):
    train_loss = train_epoch(transformer, optimizer, dataset)
    val_loss = evaluate(transformer, dataset)

    if scheduler is not None:
        scheduler.step()

    metrics = {
        "train loss": train_loss, 
        "val loss": val_loss,
    }
    wandb.log(metrics)

    print(f"""
        ======================================
        Epoch {epoch}: \n
        \t Train loss: {train_loss:.3f}, \n
        \t Val loss: {val_loss:.3f}, \n
    """)


In [ ]:
torch.save(transformer.state_dict(), 'transformer.model')
torch.save(optimizer.state_dict(), 'transformer.opt')

In [ ]:
text2ids = dataset.text2ids
ids2text = dataset.ids2text

with open(CONFIG.test_file, 'r', encoding="utf-8") as f_input:
    with open('solution.txt', 'w', encoding="utf-8") as f_output:
        for sentence in f_input.readlines():
            print(translate(transformer, sentence, text2ids, ids2text).strip(" "), file=f_output)